In [ ]:
#https://serpstack.com/ google suche aus Keywordliste in GoogleSheets
import pandas as pd
import requests
from datetime import datetime
import urllib.parse

#use creds to create a client to interact with the Google Drive API
keywordlist = pd.read_csv(r'keywords_serpstack.csv',sep=';')

print(len(keywordlist), "Keywordkombinationen werden per API gesucht.")

df = []

for i in range(0,len(keywordlist)):
    print(str(keywordlist[i]))
    #Parameter
    url = 'http://api.serpstack.com/search?'
    access_key = 'fecfdc248f2fcf66ca6541ea01eb7801'
    query = urllib.parse.quote(str(keywordlist[i])) #die Anfrage muss in eine URL umgewandelt werden
    num = 100    
    # gl: "de" oder "us" // hl: "en" oder "de"
    gl= "us"
    hl= "en"
    #page = 1 #muss wenn noch integriert werden
    
    api_result = requests.get(url+'access_key='+access_key+'&query='+query+'&num='+str(num)+'&gl='+gl+'&hl='+hl)
    api_response = api_result.json()

    if "organic_results" in api_response:
        #es kommt vor, dass zwar OrganicResults vorhanden ist aber als leere Tabelle
        if len(api_response['organic_results']) > 0:

            #Daten werden aus Serpstack übertragen
            data = pd.DataFrame(api_response['organic_results'])
            data = pd.concat([data, pd.DataFrame([keywordlist[i]] * len(data), columns=['keywords'])], axis=1, ignore_index=False)

            if i==0:
                df = data[["title","domain","url","snippet","keywords"]]
            else: 
                df = pd.concat([df, data[["title","domain","url","snippet","keywords"]]], ignore_index=True)
                df.drop_duplicates(subset ="url", keep = "first", inplace = True)
                df = df.reset_index(drop=True)
            print("Nach Durchgang", i, "Ergebnisse", len(df))
        else:
            print("Nach Durchgang", i, "Ergebnisse", len(df), "unverändert")
    else:
        print("Nach Durchgang", i, "Ergebnisse", len(df), "unverändert")


#Datum zu den Serpstack Suchresultaten hinzufügen
df = pd.concat([df, pd.DataFrame([datetime.now().strftime("%d/%m/%Y %H:%M")] * len(df), columns=['timestamp'])], axis=1, ignore_index=False)
print(len(df), "Suchergebnisse durch Serpstack insgesamt")
